In [2]:
text = 'My wife likes to watch baseball games and my daughter likes to watch baseball games too.'


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
print(cvect.fit_transform([text]).toarray())
print(cvect.vocabulary_)

[[1 2 1 2 2 2 2 1 2 1]]
{'my': 5, 'wife': 9, 'likes': 4, 'to': 6, 'watch': 8, 'baseball': 1, 'games': 3, 'and': 0, 'daughter': 2, 'too': 7}


In [6]:
ms = ['to', 'and', 'my']
cvect = CountVectorizer(stop_words=ms)
print(cvect.fit_transform([text]).toarray())
print(cvect.vocabulary_)

[[2 1 2 2 1 2 1]]
{'wife': 6, 'likes': 3, 'watch': 5, 'baseball': 0, 'games': 2, 'daughter': 1, 'too': 4}


In [8]:
cvect = CountVectorizer(stop_words='english')
print(cvect.fit_transform([text]).toarray())
print(cvect.vocabulary_)

[[2 1 2 2 2 1]]
{'wife': 5, 'likes': 3, 'watch': 4, 'baseball': 0, 'games': 2, 'daughter': 1}


In [9]:
from nltk.corpus import stopwords
sw = stopwords.words('english')
cvect = CountVectorizer(stop_words=sw)
print(cvect.fit_transform([text]).toarray())
print(cvect.vocabulary_)

[[2 1 2 2 2 1]]
{'wife': 5, 'likes': 3, 'watch': 4, 'baseball': 0, 'games': 2, 'daughter': 1}


In [10]:
def get_word(index, voca):
    for key, val in voca.items():
        if val == index:
            return key
        
print(get_word(4, cvect.vocabulary_))

watch


In [11]:
text = ['My wife likes to watch baseball games and my daughter likes to watch baseball games too.',
        'My wife likes to play baseball']

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
print(tvect.fit_transform(text).toarray())
print(tvect.vocabulary_)

[[0.38649245 0.27160082 0.54320165 0.38649245 0.         0.54320165
  0.19324622]
 [0.44832087 0.         0.         0.44832087 0.63009934 0.
  0.44832087]]
{'wife': 6, 'likes': 3, 'watch': 5, 'baseball': 0, 'games': 2, 'daughter': 1, 'play': 4}


In [13]:
cvect = CountVectorizer(stop_words='english')
print(cvect.fit_transform(text).toarray())
print(cvect.vocabulary_)

[[2 1 2 2 0 2 1]
 [1 0 0 1 1 0 1]]
{'wife': 6, 'likes': 3, 'watch': 5, 'baseball': 0, 'games': 2, 'daughter': 1, 'play': 4}


In [14]:
text = ['I work at google.', 'I google at work.']
cvect = CountVectorizer()
print(cvect.fit_transform(text).toarray())
print(cvect.vocabulary_)

[[1 1 1]
 [1 1 1]]
{'work': 2, 'at': 0, 'google': 1}


In [15]:
cvect = CountVectorizer(ngram_range=(1,2))
print(cvect.fit_transform(text).toarray())
print(cvect.vocabulary_)

[[1 1 0 1 0 1 1]
 [1 0 1 1 1 1 0]]
{'work': 5, 'at': 0, 'google': 3, 'work at': 6, 'at google': 1, 'google at': 4, 'at work': 2}


In [16]:
cvect.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 2),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [17]:
tvect.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

In [18]:
import numpy as np
import pandas as pd
df = pd.read_csv('../data/labeledTrainData.tsv', sep='\t', quoting=3)
df.head()


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [19]:
df.review = df.review.str.replace('<br />', ' ')
df.review = df.review.str.replace('[^A-Za-z]', ' ', regex=True)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review.values, df.sentiment.values, stratify=df.sentiment.values, 
    test_size=0.2, random_state=2023
)
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=int64), array([10000, 10000], dtype=int64))

In [22]:
cvect = CountVectorizer(stop_words='english')
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((20000, 66602), (5000, 66602))

In [23]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=2023, max_iter=500)
%time lrc.fit(X_train_cv, y_train)

CPU times: total: 5.28 s
Wall time: 5.28 s


LogisticRegression(max_iter=500, random_state=2023)

In [25]:
lrc.score(X_test_cv, y_test)

0.8786

In [26]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape

((20000, 1455899), (5000, 1455899))

In [28]:
lrc2 = LogisticRegression(random_state=2023, max_iter=500)
%time lrc2.fit(X_train_cv2, y_train)
lrc2.score(X_test_cv2, y_test)

CPU times: total: 55.8 s
Wall time: 49.7 s


0.8896

In [29]:
import joblib
joblib.dump(cvect2, 'model/imdb_cvect_2.pkl')
joblib.dump(lrc2, 'model/imdb_lrc2.pkl')

['model/imdb_lrc2.pkl']

In [30]:
new_cvect = joblib.load('model/imdb_cvect_2.pkl')
new_lrc = joblib.load('model/imdb_lrc2.pkl')

In [31]:
review = ['''
I was very much disappointed by this flat action movie and its predictable ending. I am a fan of the old Mission:Impossible series of the 60s and 80s and therefore I think the plot is ridiculous at best. Why should Jim Phelps do what he did? He was always loyal through the many episodes of the series and there he could have gotten much more money if he had betrayed his team.

The reason why Peter Graves (Jim Phelps) did not star in this movie is because he did not agree with what I have just said.

Anyway this movie is NOT for fans of the series because there is nothing left of the teamwork spirit of the series. It is a one man show for Tom Cruise.
''',
'''
This is, without a doubt, one of my favorite films of all time! I'll never forget watching this film for the first time with a good buddy of mine, afterward we couldn't stop talking about it and spent a great deal of time explaining plot points to each other. We finally decided that we just had to see it again, so we did and all of our questions were answered and our theories proven correct.

The story is nothing less than superb! Every time you think you have the movie figured out they throw you for another loop, but not too much as to get you irritated trying to figure out the plot. This is most definitely a film that deserves at least two viewings before you can truly understand and appreciate the story. The characters are all excellent as well, although I was sad to see Jack Harmen (Emilio Estevez) get killed off so quickly, I liked his character.

The cast is extraordinary! Tom Cruise plays Ethan Hunt perfectly! Jon Voight was the perfect choice for Jim Phelps. Emmanuelle Beart was very good in her role. Henry Czerny was superb as Kittridge. Jean Reno was an excellent addition to the cast. Ving Rhames was a very nice touch and really added a lot to the film. Kristin Scott Thomas was lovely as always, although played a somewhat minor role in the greater scheme of things. Vanessa Redgrave was another nice addition to the cast. And finally, Emilio Estevez (as I mentioned above), played a small role, and played it quite well.

I can see why some of the big fans of the show wouldn't like this film due to certain plot points that I can't give away, so if you are a big fan of the show, be forewarned, you may have some issues with the film. Personally, I've never seen a single episode of the old television show, so I had absolutely no frame of reference. Which, I believe, put me in a better position to appreciate the story.

I feel that I have to mention the action scenes in this film! SPECTACULAR!!! The scene where Kittridge and Hunt are talking in the restaurant...just AWESOME! The entire last 20 minutes of the film...UNBELIEVABLE!!! The filming, the action, the special effects and stunts alone make this film worth watching (but luckily, there is so much more to appreciate).

If you are a fan of Tom Cruise, or just crime/mystery/action films in general, be sure to check this one out (at least twice). This is honestly one of my top 20 films of all time, I truly hope that you will enjoy this film. Thanks for reading,

-Chris

'''          
]

In [46]:
import re
review = map(lambda x: re.sub('[^A-Za-z]', ' ', x), review)

In [33]:
review_cv = new_cvect.transform(review)
review_cv.shape

(2, 1455899)

In [34]:
new_lrc.predict(review_cv)

array([0, 1], dtype=int64)

In [35]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,With all this stuff going down at the moment ...
1,"""2381_9""",1,The Classic War of the Worlds by Timothy ...
2,"""7759_3""",0,The film starts with a manager Nicholas Bell...
3,"""3630_4""",0,It must be assumed that those who praised thi...
4,"""9495_8""",1,Superbly trashy and wondrously unpretentious ...


In [36]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

In [37]:
tvect = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
nb = MultinomialNB()
pipeline = Pipeline([('TVECT', tvect), ('NB', nb)])

In [38]:
%time pipeline.fit(X_train, y_train)

CPU times: total: 12.5 s
Wall time: 13 s


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('NB', MultinomialNB())])

In [39]:
pipeline.score(X_test, y_test)

0.8804

In [40]:
lrc = LogisticRegression(random_state=2023)
pipeline = Pipeline([('TVECT', tvect), ('LRC', lrc)])
%time pipeline.fit(X_train, y_train)

CPU times: total: 33.7 s
Wall time: 33.3 s


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('LRC', LogisticRegression(random_state=2023))])

In [41]:
pipeline.score(X_test, y_test)

0.8818

In [54]:
from sklearn.model_selection import GridSearchCV
params = {
    'TVECT__max_df': [100, 500],
    'LRC__C' : [1, 10]
}

grid_pipe = GridSearchCV(
    pipeline, params, scoring='accuracy', cv=3
)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 4min 40s
Wall time: 4min 45s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TVECT',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('LRC',
                                        LogisticRegression(random_state=2023))]),
             param_grid={'LRC__C': [1, 10], 'TVECT__max_df': [100, 500]},
             scoring='accuracy')

In [55]:
grid_pipe.best_params_

{'LRC__C': 10, 'TVECT__max_df': 500}

In [56]:
best_pipe = grid_pipe.best_estimator_
best_pipe.score(X_test, y_test)

0.89

In [60]:
review = ['''
I was very much disappointed by this flat action movie and its predictable ending. I am a fan of the old Mission:Impossible series of the 60s and 80s and therefore I think the plot is ridiculous at best. Why should Jim Phelps do what he did? He was always loyal through the many episodes of the series and there he could have gotten much more money if he had betrayed his team.

The reason why Peter Graves (Jim Phelps) did not star in this movie is because he did not agree with what I have just said.

Anyway this movie is NOT for fans of the series because there is nothing left of the teamwork spirit of the series. It is a one man show for Tom Cruise.
''',
'''
This is, without a doubt, one of my favorite films of all time! I'll never forget watching this film for the first time with a good buddy of mine, afterward we couldn't stop talking about it and spent a great deal of time explaining plot points to each other. We finally decided that we just had to see it again, so we did and all of our questions were answered and our theories proven correct.

The story is nothing less than superb! Every time you think you have the movie figured out they throw you for another loop, but not too much as to get you irritated trying to figure out the plot. This is most definitely a film that deserves at least two viewings before you can truly understand and appreciate the story. The characters are all excellent as well, although I was sad to see Jack Harmen (Emilio Estevez) get killed off so quickly, I liked his character.

The cast is extraordinary! Tom Cruise plays Ethan Hunt perfectly! Jon Voight was the perfect choice for Jim Phelps. Emmanuelle Beart was very good in her role. Henry Czerny was superb as Kittridge. Jean Reno was an excellent addition to the cast. Ving Rhames was a very nice touch and really added a lot to the film. Kristin Scott Thomas was lovely as always, although played a somewhat minor role in the greater scheme of things. Vanessa Redgrave was another nice addition to the cast. And finally, Emilio Estevez (as I mentioned above), played a small role, and played it quite well.

I can see why some of the big fans of the show wouldn't like this film due to certain plot points that I can't give away, so if you are a big fan of the show, be forewarned, you may have some issues with the film. Personally, I've never seen a single episode of the old television show, so I had absolutely no frame of reference. Which, I believe, put me in a better position to appreciate the story.

I feel that I have to mention the action scenes in this film! SPECTACULAR!!! The scene where Kittridge and Hunt are talking in the restaurant...just AWESOME! The entire last 20 minutes of the film...UNBELIEVABLE!!! The filming, the action, the special effects and stunts alone make this film worth watching (but luckily, there is so much more to appreciate).

If you are a fan of Tom Cruise, or just crime/mystery/action films in general, be sure to check this one out (at least twice). This is honestly one of my top 20 films of all time, I truly hope that you will enjoy this film. Thanks for reading,

-Chris

'''          
]

In [61]:
import re
review = map(lambda x: re.sub('[^A-Za-z]', ' ', x), review)


In [62]:
best_pipe.predict(review)

array([1, 1], dtype=int64)

In [63]:
url = 'https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv'

In [65]:
df = pd.read_csv(url, encoding='latin1')
df = df[['v1', 'v2']]

In [68]:
df.drop_duplicates(subset=['v2'], inplace=True)


In [69]:
df.v1 = df.v1.replace(['ham', 'spam'], [0, 1])

In [72]:
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [73]:
df.v2 = df.v2.str.replace('[^A-Za-z]', ' ', regex=True)

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.v2.values, df.v1.values, stratify=df.v1.values, test_size=0.2, random_state=2023
)


In [80]:
from sklearn.ensemble import RandomForestClassifier
params = {
    'CVECT__ngram_range':[(1, 1)],
    'RFC__max_depth':[114, 115, 116]
}

In [81]:
cvect = CountVectorizer(stop_words='english')
rfc = RandomForestClassifier(random_state=2023)
pipeline = Pipeline([('CVECT', cvect), ('RFC', rfc)])
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 20.9 s
Wall time: 21.9 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT',
                                        CountVectorizer(stop_words='english')),
                                       ('RFC',
                                        RandomForestClassifier(random_state=2023))]),
             param_grid={'CVECT__ngram_range': [(1, 1)],
                         'RFC__max_depth': [114, 115, 116]},
             scoring='accuracy')

In [82]:
grid_pipe.best_params_

{'CVECT__ngram_range': (1, 1), 'RFC__max_depth': 115}

In [83]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.9671179883945842

In [95]:
params = {
    'TVECT__max_df':[100, 500],
    'RFC__max_depth':[116, 120]
}

tvect = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
nb = MultinomialNB()
pipeline = Pipeline([('TVECT', tvect), ('RFC', rfc)])
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 41.2 s
Wall time: 43.1 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TVECT',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('RFC',
                                        RandomForestClassifier(random_state=2023))]),
             param_grid={'RFC__max_depth': [116, 120],
                         'TVECT__max_df': [100, 500]},
             scoring='accuracy')

In [96]:
grid_pipe.best_params_

{'RFC__max_depth': 116, 'TVECT__max_df': 500}

In [97]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.9584139264990329

In [100]:
review = ['''
I was very much disappointed by this flat action movie and its predictable ending. I am a fan of the old Mission:Impossible series of the 60s and 80s and therefore I think the plot is ridiculous at best. Why should Jim Phelps do what he did? He was always loyal through the many episodes of the series and there he could have gotten much more money if he had betrayed his team.

The reason why Peter Graves (Jim Phelps) did not star in this movie is because he did not agree with what I have just said.

Anyway this movie is NOT for fans of the series because there is nothing left of the teamwork spirit of the series. It is a one man show for Tom Cruise.
''',
'''
This is, without a doubt, one of my favorite films of all time! I'll never forget watching this film for the first time with a good buddy of mine, afterward we couldn't stop talking about it and spent a great deal of time explaining plot points to each other. We finally decided that we just had to see it again, so we did and all of our questions were answered and our theories proven correct.

The story is nothing less than superb! Every time you think you have the movie figured out they throw you for another loop, but not too much as to get you irritated trying to figure out the plot. This is most definitely a film that deserves at least two viewings before you can truly understand and appreciate the story. The characters are all excellent as well, although I was sad to see Jack Harmen (Emilio Estevez) get killed off so quickly, I liked his character.

The cast is extraordinary! Tom Cruise plays Ethan Hunt perfectly! Jon Voight was the perfect choice for Jim Phelps. Emmanuelle Beart was very good in her role. Henry Czerny was superb as Kittridge. Jean Reno was an excellent addition to the cast. Ving Rhames was a very nice touch and really added a lot to the film. Kristin Scott Thomas was lovely as always, although played a somewhat minor role in the greater scheme of things. Vanessa Redgrave was another nice addition to the cast. And finally, Emilio Estevez (as I mentioned above), played a small role, and played it quite well.

I can see why some of the big fans of the show wouldn't like this film due to certain plot points that I can't give away, so if you are a big fan of the show, be forewarned, you may have some issues with the film. Personally, I've never seen a single episode of the old television show, so I had absolutely no frame of reference. Which, I believe, put me in a better position to appreciate the story.

I feel that I have to mention the action scenes in this film! SPECTACULAR!!! The scene where Kittridge and Hunt are talking in the restaurant...just AWESOME! The entire last 20 minutes of the film...UNBELIEVABLE!!! The filming, the action, the special effects and stunts alone make this film worth watching (but luckily, there is so much more to appreciate).

If you are a fan of Tom Cruise, or just crime/mystery/action films in general, be sure to check this one out (at least twice). This is honestly one of my top 20 films of all time, I truly hope that you will enjoy this film. Thanks for reading,

-Chris

'''          
]

review = map(lambda x: re.sub('[^A-Za-z]', ' ', x), review)

In [101]:
grid_pipe.best_estimator_.predict(review)

array([0, 0], dtype=int64)

In [1]:
import pandas as pd

df2 = pd.read_csv('https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt', sep='\t')
df2.head()

,5,배공빠르고 굿
0,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
1,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
2,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
3,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
4,2,비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적...


In [4]:
from konlpy.tag import Okt
okt = Okt()
text = "test 아주 좋아요. !!!"

In [5]:
m = okt.morphs(text)
m

['test', '아주', '좋아요', '.', '!!!']

In [6]:
okt.nouns(text)

['아주']